<a href="https://colab.research.google.com/github/GalBuzi/colab/blob/main/inceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression

from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img
import random
import cv2
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model
from google.colab import files
import io
import os
from google.colab import drive
from sklearn.metrics import confusion_matrix,accuracy_score,log_loss

drive.mount('gdrive')

In [ ]:

train_dir = 'gdrive/My Drive/dog-breed-identification/train/'
csv_file = open('gdrive/My Drive/dog-breed-identification/labels.csv')
label = pd.read_csv(csv_file)

In [ ]:
label['path'] = train_dir + label.id + '.jpg'

fig,axes = plt.subplots(2,5,figsize = (30,10))

for ax in axes.reshape(-1,):
    rnd_idx = np.random.randint(label.index[0],label.index[-1])
    file_path = label.loc[rnd_idx,'path']
    arr = plt.imread(file_path)
    ax.imshow(arr)
    ax.set_title(label.loc[rnd_idx,'breed']+'\n'+str(arr.shape))
    ax.axis('off')

In [ ]:
# sizes_info = label.path.apply(lambda x: plt.imread(x).shape).values.tolist()
# meta_df    = pd.DataFrame(sizes_info,columns = ['height','width','channels'])
# label      = pd.merge(label,meta_df,how = 'left',left_index = True,right_index=True)
# label

In [ ]:
label['id_jpg'] = label.id+'.jpg'
breed_list = label.breed.unique()
print(label)
print(breed_list)

In [ ]:
def save_model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")

def load_model():
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("model.h5")
    return loaded_model

def confusion_matrix_predict(model, validation_generator):
    preds = model.predict(validation_generator)
    predicts = np.argmax(preds, axis=1)  # return the predicted category in each sample in test set
    print(confusion_matrix(validation_generator.index_array, predicts))
    print('model accuracy on test set is: {}%'.format(accuracy_score(validation_generator.index_array, predicts) * 100))
    sns.heatmap(confusion_matrix(validation_generator.index_array, predicts), cmap='Greens', annot=False, fmt='d')
    plt.xlabel('Prediction')
    plt.ylabel('True label')
    plt.title('Classification results on test set')
    plt.show()

def plot_history(history):

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# Train and Test split

train_df, validate_df = train_test_split(label, test_size=0.2)
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print(train_df)
print(validate_df)

print(train_df.shape, validate_df.shape)

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 50
batch_size = 32

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=18,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)


train_generator = train_datagen.flow_from_dataframe(
    train_df,
    r'gdrive/My Drive/dog-breed-identification/train',
    x_col='id_jpg',
    y_col='breed',
    class_mode='categorical',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    color_mode = 'rgb',
    seed=42,
    shuffle=False
)

validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    r'gdrive/My Drive/dog-breed-identification/train',
    x_col='id_jpg',
    y_col='breed',
    class_mode='categorical',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    color_mode = 'rgb',
    seed=42,
    shuffle=False
)

In [ ]:
classes = train_generator.class_indices
classes

In [ ]:
pre_trained_model = inception_v3(include_top = False, weights = 'imagenet', input_shape = input_shape)

pre_trained_model.trainbale = False
for layer in pre_trained_model.layers:
    layer.trainable = False
    

model = Sequential([
    pre_trained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation ='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256,activation ='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(120,activation = 'softmax')
])

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics = ['accuracy'])

history = model.fit_generator(generator = train_generator,
                           epochs = 3,
                           steps_per_epoch  = train_generator.n//train_generator.batch_size,
                           validation_data  = validation_generator,
                           validation_steps = validation_generator.n//validation_generator.batch_size)


loss, accuracy = model.evaluate_generator(validation_generator, validation_generator.batch_size, workers=12)
print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))

In [ ]:
plot_history(history) 
confusion_matrix_predict(model,validation_generator)
save_model(model)

In [ ]:
val_pred = np.argmax(model.predict(validation_generator),axis=-1)
classes = {x:y for y,x in classes.items()}

fig,axes = plt.subplots(2,5,figsize = (30,10))

for ax in axes.reshape(-1,):
    rnd_idx = np.random.randint(0,len(validation_generator.filepaths))
    arr = plt.imread(validation_generator.filepaths[rnd_idx])
    ax.imshow(arr)
    breed_true = classes[validation_generator.classes[rnd_idx]]
    breed_pred = classes[val_pred[rnd_idx]]
    
    if breed_true != classes[val_pred[rnd_idx]]:
        ax.set_title(breed_true+'\n'+ breed_pred, color = 'red',fontsize = 15)
    else:
        ax.set_title(breed_true+'\n'+ breed_pred, color = 'black',fontsize = 15)

    ax.axis('off')